In [1]:
import gmsh

def create_1d_mesh(filename="model_1d.msh"):
    gmsh.initialize()
    gmsh.model.add("Core_1D")

    # 1. 定义几何点 (长度 0.1m)
    lc = 0.002  # 网格尺寸 2mm
    gmsh.model.geo.addPoint(0, 0, 0, lc, 1)
    gmsh.model.geo.addPoint(0.1, 0, 0, lc, 2)

    # 2. 定义直线
    gmsh.model.geo.addLine(1, 2, 1)

    gmsh.model.geo.synchronize()

    # 3. 添加物理标记 (非常重要：对应你代码里的 ID)
    # 边界标记：1=左边界(Upstream), 2=右边界(Downstream)
    gmsh.model.addPhysicalGroup(0, [1], 1) 
    gmsh.model.addPhysicalGroup(0, [2], 2)
    # 材料标记：0=岩石区域 (De_ref_dict={0: ...})
    gmsh.model.addPhysicalGroup(1, [1], 0)

    # 4. 生成网格
    gmsh.model.mesh.generate(1)
    gmsh.write(filename)
    gmsh.finalize()
    print(f"Mesh saved as {filename}")

if __name__ == "__main__":
    create_1d_mesh()

Info    : Meshing 1D...
Info    : Meshing curve 1 (Line)
Info    : Done meshing 1D (Wall 0.00302531s, CPU 0.002358s)
Info    : 51 nodes 52 elements
Info    : Writing 'model_1d.msh'...
Info    : Done writing 'model_1d.msh'
Mesh saved as model_1d.msh


In [1]:
# 导入你那个 ADE_DG-Temp.py 里的类
# 假设你的文件名叫 ADE_DG_Temp.py
from ADE_DG_Temp import ADE_DG 
import numpy as np

# 1. 初始化模型 (读取刚才生成的网格)
model = ADE_DG("model_1d.msh", deg=1)

# 2. 设置假设参数
model.set_De({0: 1.0e-10})          # 参考温度下的扩散系数
model.set_porosity({0: 0.2})        # 孔隙度 20%
model.set_darcy_flux([1.0e-5, 0])   # 极慢的水流速度 (m/s)
model.set_T_end(720.0)             # 模拟 2 小时 (7200秒)
model.set_dt(10.0)                  # 初始步长 10秒

# 3. 设置边界位置 (对应 Gmsh 里的 0 和 0.1)
model.set_upstream_boundary(id=1, location=0.0)
model.set_downstream_boundary(id=2, location=0.1)
model.set_upstream_reservoir_concentration(1.0)
model.set_downstream_reservoir_concentration(0.0)

# 4. 定义【变温函数】
# 4. 定义【阶梯变温函数】：20, 30, 40, 50, 60 °C
def my_temp_schedule(t):
    # 2小时内从20度线性升到60度
    start_temp = 20
    end_temp = 60
    duration = 7200
    temp_c = start_temp + (end_temp - start_temp) * (t / duration)
    return temp_c + 273.15
# 或者用更简洁的写法（数学计算）：
# def my_temp_schedule(t):
#     # 每 1440 秒增加 10 度，最高到 60 度
#     step = min(int(t / 1440), 4) 
#     temp_c = 20 + step * 10
#     return temp_c + 273.15

# 5. 运行自适应求解
# 此时内部会通过 update_De_from_Temp_arrhenius 自动改变 De
final_u, df = model.solve_adaptive_timesteps(Temp_input=my_temp_schedule)

# 6. 检查结果
print("Simulation done!")
print(df.tail()) # 看看最后几行，注意 dt 是否在升温瞬间变小了

INFO:root:INFO: init with: model_1d.msh deg 1
INFO:root:INFO: mesh read! mesh dimension: 1 boundary dimension 0 geometry dimension: 2
INFO:root:INFO: De set via dictionary {0: 1e-10} default: 1.0e-11 
INFO:root:porosity set via dictionary {0: 0.2} default: 1.0
INFO:root:Darcy flux is set to: [1e-05, 0] for the whole domain.
INFO:root:INFO: Adaptive time loop starts with error tolerance: 0.001 update_reservoir_concentrations: True
INFO:root:INFO: t dt error est     0.0000 |     6.5466 |  0.0006678
  df = pd.concat([df, new_row], ignore_index=True)



Info    : Reading 'model_1d.msh'...
Info    : 3 entities
Info    : 51 nodes
Info    : 52 elements
Info    : Done reading 'model_1d.msh'


INFO:root:INFO: t dt error est     6.5466 |     7.6000 |  0.0001326
INFO:root:INFO: t dt error est    14.1466 |    19.7993 |  0.0007503
INFO:root:INFO: t dt error est    33.9459 |    21.6852 |  0.0005130
INFO:root:INFO: t dt error est    55.6311 |    28.7215 |  0.0008031
INFO:root:INFO: t dt error est    84.3525 |    30.4053 |  0.0007821
INFO:root:INFO: t dt error est   114.7579 |    32.6157 |  0.0007835
INFO:root:INFO: t dt error est   147.3735 |    34.9575 |  0.0008186
INFO:root:INFO: t dt error est   182.3311 |    36.6535 |  0.0008312
INFO:root:INFO: t dt error est   218.9846 |    38.1402 |  0.0008396
INFO:root:INFO: t dt error est   257.1248 |    39.4894 |  0.0008464
INFO:root:INFO: t dt error est   296.6141 |    40.7207 |  0.0008520
INFO:root:INFO: t dt error est   337.3348 |    41.8511 |  0.0008566
INFO:root:INFO: t dt error est   379.1860 |    42.8972 |  0.0008604
INFO:root:INFO: t dt error est   422.0831 |    43.8723 |  0.0008636
INFO:root:INFO: t dt error est   465.9554 |    4

Simulation done!
          time         dt  Upstream_concentration  Downstream_concentration  \
16  510.742267  45.648903                     1.0             -1.006130e-37   
17  556.391170  46.465113                     1.0             -7.243175e-37   
18  602.856283  47.240821                     1.0              1.472599e-35   
19  650.097104  47.980483                     1.0              3.547356e-35   
20  698.077587  21.922413                     1.0              8.059060e-34   

    Upstream_flux  Downstream_flux  
16   3.047568e-18     2.353684e-39  
17   4.774274e-19     1.342307e-38  
18   7.216077e-20    -3.270543e-37  
19   1.053811e-20    -4.324168e-37  
20   4.626675e-21    -3.514360e-35  
